### Simulating Quantum Teleportation Usig Cirq

In [18]:
import cirq

def quantum_teleportation(qubit_to_send_op='H', num_copies=100):
    # Define 3 qubits: Q1 (Alice's qubit to send), Q2 (Alice's entangled qubit), Q3 (Bob's entangled qubit)
    Q1, Q2, Q3 = [cirq.LineQubit(i) for i in range(3)]
    
    # Create a circuit
    circuit = cirq.Circuit()
    
    """
    Q1 : Alice's state qubit to be sent to Bob
    Q2 : Alice's entangled qubit
    Q3 : Bob's entangled qubit
    Set the initial state of Q1 based on qubit_to_send_op
    """
    if qubit_to_send_op == 'H':
        circuit.append(cirq.H(Q1))
    elif qubit_to_send_op == 'X':
        circuit.append(cirq.X(Q1))
    elif qubit_to_send_op == 'Y':
        circuit.append(cirq.Y(Q1))
    elif qubit_to_send_op == 'Z':
        circuit.append(cirq.Z(Q1))
    elif qubit_to_send_op == 'I':
        circuit.append(cirq.I(Q1))
    else:
        raise NotImplementedError("Operator not implemented")
    
    # Entangle Alice and Bob's qubits: Q2 and Q3
    circuit.append(cirq.H(Q2))
    circuit.append(cirq.CNOT(Q2, Q3))
    
    # CNOT Alice's data qubit Q1 with Alice's entangled qubit Q2
    circuit.append(cirq.CNOT(Q1, Q2))
    
    # Transform Alice's data qubit Q1 to +/- basis
    circuit.append(cirq.H(Q1))
    
    # Measure Alice's qubits Q1 and Q2
    circuit.append(cirq.measure(Q1, Q2))
    
    # Apply conditional operations on Bob's qubit Q3 based on Alice's measurements
    circuit.append(cirq.CNOT(Q2, Q3))
    circuit.append(cirq.CZ(Q1, Q3))
    
    # Measure Bob's final qubit
    circuit.append(cirq.measure(Q3, key='bobs_qubit'))
    
    # Print the circuit
    print("Quantum Teleportation Circuit:")
    print(circuit)
    
    # Simulate the circuit
    sim = cirq.Simulator()
    output = sim.run(circuit, repetitions=num_copies)
    
    # Print measurement histogram for Bob's qubit
    print("Measurement Output (Bob's qubit Q3):")
    print(output.histogram(key='bobs_qubit'))

# Run the function if this script is executed
if __name__ == '__main__':
    quantum_teleportation(qubit_to_send_op='I')


Quantum Teleportation Circuit:
0: ───I───────@───H───M───────@─────────────────────
              │       │       │
1: ───H───@───X───────M───@───┼─────────────────────
          │               │   │
2: ───────X───────────────X───@───M('bobs_qubit')───
Measurement Output (Bob's qubit Q3):
Counter({0: 100})


### Qiskit

In [ ]:
"""
This is an implementation of quantum teleportation using Qiskit.
Here, Alice wants to send the state of her qubit (Q0) to Bob's qubit (Q2) using an entangled pair (Q1 and Q2).
The teleportation protocol involves entangling Alice's qubit with her half of the entangled pair, measuring her qubits,
and applying conditional operations on Bob's qubit based on Alice's measurement results.
"""

from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

def quantum_teleportation(qubit_to_send_op='h', num_copies=1000):
    # Create a Quantum Circuit with 3 qubits and 3 classical bits
    qc = QuantumCircuit(3, 3)

    # Prepare Alice's qubit Q0
    qubit_to_send_op = qubit_to_send_op.lower()
    if qubit_to_send_op == 'h':
        qc.h(0)
    elif qubit_to_send_op == 'x':
        qc.x(0)
    elif qubit_to_send_op == 'y':
        qc.y(0)
    elif qubit_to_send_op == 'z':
        qc.z(0)
    elif qubit_to_send_op == 'i':
        pass
    else:
        raise NotImplementedError("Operator not implemented")
    
    # Entangle Alice's Q1 and Bob's Q2
    qc.h(1)
    qc.cx(1, 2)
    
    # Alice applies teleportation operations
    qc.cx(0, 1)
    qc.h(0)
    
    # Measure Alice's qubits into classical bits 0 and 1
    qc.measure([0, 1], [0, 1])
    
    # Directly apply CX and CZ on Bob's qubit (ideal case)
    qc.cx(1, 2)
    qc.cz(0, 2)
    
    # Measure Bob's qubit into classical bit 2
    qc.measure(2, 2)
    
    # Print the circuit
    print("Quantum Teleportation Circuit (CX & CZ directly applied):")
    print(qc.draw(output='text'))
    
    # Simulate the circuit
    simulator = AerSimulator()
    result = simulator.run(qc, shots=num_copies).result()
    
    # Get measurement counts for all qubits
    counts = result.get_counts()
    print("\nMeasurement counts (all qubits):")
    print(counts)
    
    # Extract Bob's qubit (Q2) counts
    bob_counts = {'0': 0, '1': 0}
    for outcome, count in counts.items():
        # Q2 is the first character in the bitstring (Q2, Q1, Q0)
        # Qiskit uses big-endian ordering in counts
        bob_bit = outcome[0]
        bob_counts[bob_bit] += count 
    
    print("\nBob's qubit measurement counts:")
    print(bob_counts)

# Run the function
if __name__ == "__main__":
    quantum_teleportation(qubit_to_send_op='i', num_copies=1000)


Quantum Teleportation Circuit (CX & CZ directly applied):
                    ┌───┐┌─┐           
q_0: ────────────■──┤ H ├┤M├──────■────
     ┌───┐     ┌─┴─┐└┬─┬┘└╥┘      │    
q_1: ┤ H ├──■──┤ X ├─┤M├──╫───■───┼────
     └───┘┌─┴─┐└───┘ └╥┘  ║ ┌─┴─┐ │ ┌─┐
q_2: ─────┤ X ├───────╫───╫─┤ X ├─■─┤M├
          └───┘       ║   ║ └───┘   └╥┘
c: 3/═════════════════╩═══╩══════════╩═
                      1   0          2 

Measurement counts (all qubits):
{'011': 229, '010': 275, '001': 242, '000': 254}

Bob's qubit measurement counts:
{'0': 1000, '1': 0}
